In [3]:
import os 

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

azure_ai_project = os.environ.get("AZURE_PROJECT_ENDPOINT")
azure_openai_deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")
azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.environ.get("AZURE_OPENAI_API_KEY")  
azure_openai_api_version = os.environ.get("AZURE_OPENAI_API_VERSION")  

In [8]:
from azure.ai.evaluation import evaluate, GroundednessEvaluator, F1ScoreEvaluator, FluencyEvaluator

model_config = {
    "azure_endpoint": azure_openai_endpoint,
    "azure_deployment": azure_openai_deployment,
    "api_key": azure_openai_api_key,
}

groundedness_eval = GroundednessEvaluator(model_config=model_config)
f1_eval = F1ScoreEvaluator(threshold=0.6)
fluency_eval = FluencyEvaluator(model_config=model_config)

# Define a simple answer length evaluator
def answer_length(response, **kwargs):
    return {"answer_length": len(response)}

result = evaluate(
    data="data.jsonl", # Provide your data here:
    evaluators={
        "groundedness": groundedness_eval,
        "answer_length": answer_length,
        "f1-score": f1_eval,
        "fluency": fluency_eval
    },
    # Column mapping:
    evaluator_config={
        "groundedness": {
            "column_mapping": {
                "query": "${data.query}",
                "context": "${data.context}",
                "response": "${data.response}"
            } 
        }
    },
    # Optionally, provide your Azure AI Foundry project information to track your evaluation results in your project portal.
    azure_ai_project = azure_ai_project,
    # Optionally, provide an output path to dump a JSON file of metric summary, row-level data, and the metric and Azure AI project URL.
    output_path="./myevalresults.json"
)